# Capstone Project

# This notebook will be mainly used for the capstone project.

In [4]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto.

In [23]:
# Importing Libraries

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
from bs4 import BeautifulSoup

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
beautiful_soup = BeautifulSoup(source, 'html5lib')
postal_code_list = []
borough_list = []
neighborhood_list = []

In [24]:
# Reading Table

table = beautiful_soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [25]:
# Creating Dataframe

df = pd.DataFrame(row)
for row in beautiful_soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postal_code_list.append(cells[0].text)
        borough_list.append(cells[1].text)
        neighborhood_list.append(cells[2].text.rstrip('\n'))
df = pd.DataFrame({"PostalCode": postal_code_list,
                   "Borough": borough_list,
                   "Neighborhood": neighborhood_list})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [31]:
# Grouping Neighborhoods (only process the cells that have an assigned borough)

df_cleaned = df[df.Borough != "Not assigned"].reset_index(drop = True)
df_cleaned.head()
df_grouped = df_cleaned.groupby(["PostalCode", "Borough"], as_index = False).agg(lambda x: ",".join(x))
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
# Importing Geospatial Data

df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
# Merging Dataframes

df_merged = pd.merge(df_grouped, df_geo, on = "PostalCode")
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [39]:
# Printing the Number of Rows

df_merged.shape

(103, 5)